In [1]:
from dataset_generator import TimeWindowDatasetGenerator

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.layers import LSTM
from sklearn.impute import SimpleImputer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2024-03-14 17:10:57.393852: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-14 17:10:57.456082: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-14 17:10:57.686002: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-14 17:10:58.586828: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

data_generator = TimeWindowDatasetGenerator()
labeled_data = data_generator.get_labelled_timewindow_dataframe(student_data_filepath='task/student_data.csv',
                                                                time_window=10,
                                                                label_feature='affect',
                                                                exclude_after_notification=False,
                                                                exclude_after_engagement=False)


labeled_data
#print(labeled_data)


,sessionId,timestamp,x,y,z,ppgValue,hr,hrIbi,hrStatus,ibiStatus,notification,engagement,affect,context,label_id,label
12763,1,493272,884.0,1564.0,3767.0,2103379.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
12764,1,493311,900.0,1518.0,3782.0,2104145.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
12765,1,493351,894.0,1553.0,3734.0,2104953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
12766,1,493391,907.0,1488.0,3729.0,2105398.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
12767,1,493431,861.0,1559.0,3731.0,2105114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,RELAXED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8101814,58,6184903,453.0,552.0,4126.0,2126027.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240,HAPPY
8101815,58,6184943,494.0,607.0,4059.0,2126361.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240,HAPPY
8101816,58,6184983,695.0,841.0,4124.0,2127234.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240,HAPPY
8101817,58,6185023,497.0,591.0,4329.0,2128049.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240,HAPPY


In [3]:
#for i in range(1, 241):
#   sequences_group_i = labeled_data.groupby('label_id').get_group(i)
#    print(f"Shape of group {i}: {sequences_group_i.shape}")

In [4]:
labeled_data.describe()
labeled_data.to_csv("task/labeled_data.csv")

In [5]:

def preprocess_data(data):
  
    data = data.drop(columns=[ 'notification', 'engagement','context','affect','hr','hrIbi','hrStatus','ibiStatus',])


    scaler = StandardScaler()
    numerical_cols = ['sessionId','timestamp','x', 'y', 'z', 'ppgValue']
    data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

    label_encoder = LabelEncoder()
    data['label'] = label_encoder.fit_transform(data['label'])

    return data

processed_data = preprocess_data(labeled_data)
processed_data

,sessionId,timestamp,x,y,z,ppgValue,label_id,label
12763,-2.093918,-1.249012,0.767907,0.827578,0.280375,-0.196957,1,2
12764,-2.093918,-1.248996,0.781188,0.800646,0.291041,-0.193802,1,2
12765,-2.093918,-1.248979,0.776207,0.821138,0.256910,-0.190475,1,2
12766,-2.093918,-1.248962,0.786998,0.783081,0.253355,-0.188643,1,2
12767,-2.093918,-1.248945,0.748816,0.824651,0.254777,-0.189812,1,2
...,...,...,...,...,...,...,...,...
8101814,1.480461,1.157004,0.410159,0.235065,0.535647,-0.103694,240,1
8101815,1.480461,1.157021,0.444190,0.267266,0.488006,-0.102319,240,1
8101816,1.480461,1.157037,0.611029,0.404271,0.534225,-0.098724,240,1
8101817,1.480461,1.157054,0.446681,0.257899,0.679993,-0.095368,240,1


In [7]:
grouped = processed_data.groupby('label_id')

def pad_sequence(sequence):
    padded_sequence = np.pad(sequence, (0, 265 - len(sequence)), 'constant', constant_values=np.nan)
    return padded_sequence

# Create an empty DataFrame to store padded sequences
padded_df = pd.DataFrame(columns=processed_data.columns)

# Iterate over groups
for label_id, group in grouped:
    # Get sequences from each group
    sequences = group.drop(['sessionId', 'timestamp', 'label_id', 'label'], axis=1).values
    
    # Pad each sequence
    padded_sequences = np.apply_along_axis(pad_sequence, 0, sequences)
    
    # Create DataFrame with padded sequences and add 'label_id' and 'label' columns
    padded_group_df = pd.DataFrame(padded_sequences, columns=processed_data.columns[2:-2])
    padded_group_df['label_id'] = label_id
    padded_group_df['label'] = group['label'].iloc[0]  # Assuming label is the same for all rows in a group
    
    # Append padded group DataFrame to the final DataFrame
    padded_df = pd.concat([padded_df, padded_group_df], ignore_index=True)

# Now padded_df contains the padded sequences with each group having a length of 265


In [18]:
from sklearn.impute import KNNImputer
X = padded_df.drop(['sessionId', 'label_id', 'label'], axis=1).values
y = padded_df['label'].values

X = X.astype(np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Reshape sequences for LSTM input
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Convert labels to numeric format
label_mapping = {label: idx for idx, label in enumerate(np.unique(y))}
numeric_y_train = np.array([label_mapping[label] for label in y_train])
numeric_y_test = np.array([label_mapping[label] for label in y_test])

# Reshape X_train and X_test to 2D arrays
num_samples_train, num_timesteps_train, num_features_train = X_train.shape
X_train_reshaped = X_train.reshape((num_samples_train * num_timesteps_train, num_features_train))

num_samples_test, num_timesteps_test, num_features_test = X_test.shape
X_test_reshaped = X_test.reshape((num_samples_test * num_timesteps_test, num_features_test))

# Apply imputation
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train_reshaped)
X_test_imputed = imputer.transform(X_test_reshaped)

#knn_imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors as needed

# Apply KNN imputation
#X_train_imputed = knn_imputer.fit_transform(X_train_reshaped)
#X_test_imputed = knn_imputer.transform(X_test_reshaped)

# Reshape X_train_imputed and X_test_imputed back to 3D arrays
X_train_imputed = X_train_imputed.reshape((num_samples_train, num_timesteps_train, num_features_train))
X_test_imputed = X_test_imputed.reshape((num_samples_test, num_timesteps_test, num_features_test))

# Check the shapes of the splits
print("Training set shapes:")
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("\nTesting set shapes:")
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

from tensorflow.keras.layers import Masking

# Define LSTM model with masking layer
model = Sequential()
#model.add(Masking(mask_value=np.nan, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(64,input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(7, activation='softmax'))

# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X_test_imputed, numeric_y_train, epochs=100, batch_size=32, validation_split=0.2)


Training set shapes:
X_train: (57240, 5, 1)
y_train: (57240,)

Testing set shapes:
X_test: (6360, 5, 1)
y_test: (6360,)
Epoch 1/100


/home/harsha/bbdc24-mlbananas/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4303 - loss: 1.4680 - val_accuracy: 0.4803 - val_loss: 0.9255
Epoch 2/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4663 - loss: 0.9242 - val_accuracy: 0.4686 - val_loss: 0.9131
Epoch 3/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4782 - loss: 0.9124 - val_accuracy: 0.4827 - val_loss: 0.9098
Epoch 4/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4760 - loss: 0.9037 - val_accuracy: 0.4819 - val_loss: 0.9083
Epoch 5/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4809 - loss: 0.9126 - val_accuracy: 0.4662 - val_loss: 0.9086
Epoch 6/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4705 - loss: 0.9199 - val_accuracy: 0.4646 - val_loss: 0.9094
Epoch 7/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4745 - loss: 0.9206 - val_accuracy: 0.4623 - val_loss: 0.9097
Epoch 8/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4743 - loss: 0.9114 - val_accuracy: 0.4803

In [22]:
# Make predictions on the test data
y_pred_probabilities = model.predict(X_test_imputed)

# Convert predicted probabilities to class labels
y_pred = np.argmax(y_pred_probabilities, axis=1)
# Evaluate predictions
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(numeric_y_test, y_pred)
print("Accuracy:", accuracy)

# You can also print a classification report for more detailed evaluation metrics
print(classification_report(numeric_y_test, y_pred))


  1/199 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step

199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 708us/step
Accuracy: 0.480188679245283
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       123
           1       0.47      0.41      0.44      2985
           2       0.49      0.61      0.54      3000
           3       0.00      0.00      0.00       252

    accuracy                           0.48      6360
   macro avg       0.24      0.26      0.24      6360
weighted avg       0.45      0.48      0.46      6360



/home/harsha/bbdc24-mlbananas/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/harsha/bbdc24-mlbananas/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/harsha/bbdc24-mlbananas/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz